In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy import isnan
from sklearn import svm
from sklearn import preprocessing
from scipy.stats import kurtosis, skew
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from mpl_toolkits.mplot3d import Axes3D
from sklearn.multiclass import OneVsRestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict, cross_validate, RepeatedStratifiedKFold, GridSearchCV
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve, confusion_matrix, auc, r2_score
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler, SVMSMOTE
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFE, RFECV
from numpy import savetxt, loadtxt
from numpy import mean, std


In [ ]:
stats_df = pd.read_csv('my_table_patients_stats_measure.csv') 

In [11]:
sub_all = np.arange(0,116,1).tolist()
X1 = imputation_feature_extraction(idx=sub_all)
n_features = 5
n_repeats = 1500

In [ ]:
outer_results = list()
outer_results2 = list()
outer_results3 = list()
specificity = list()

feature_ranking = np.zeros(shape=(n_repeats*5,X1.shape[1]))

tns = list()
fps = list()
tps = list()
fns = list()
#scaler = MinMaxScaler()
scaler1 = StandardScaler()

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
plt.figure(figsize=(20,20))

cv_outer = RepeatedStratifiedKFold(n_splits=5, n_repeats=n_repeats, random_state=1)
iter_ = 0
for i,(train_ix, test_ix) in enumerate(cv_outer.split(X1,y)):
    
    X1_train, X1_test = X1[train_ix, :], X1[test_ix, :]
    y_train, y_test = y[train_ix], y[test_ix]
    #age_train = age[train_ix]
    #age_test = age[test_ix]
    #gender_train = gender[train_ix]
    #gender_test = gender[test_ix]
    
    X1_train = scaler1.fit_transform(X1_train)
    X1_test = scaler1.transform(X1_test)

    model1 = svm.SVC(kernel='linear',class_weight='balanced', gamma='scale', C=1) #3 set C = 0.5 gamma='scale',
    
    sfs1 = RFE(estimator=model1, step=1, n_features_to_select=n_features)

    sfs1 = sfs1.fit(X1_train, y_train)
    
    feature_ranking[iter_,np.argwhere(sfs1.ranking_==1).ravel()] = 1

    x_train_sfs = sfs1.transform(X1_train)
    x_test_sfs = sfs1.transform(X1_test)
    #print(sfs1.ranking_)
    #print(np.argwhere(sfs1.ranking_==1))
    #x_train_sfs = np.hstack((age_train.reshape(-1,1), x_train_sfs))
    #x_test_sfs = np.hstack((age_test.reshsape(-1,1), x_test_sfs))
    
    x_train_sfs = scaler1.fit_transform(x_train_sfs)
    x_test_sfs = scaler1.transform(x_test_sfs)
    
    cv_inner = RepeatedStratifiedKFold(n_splits=5, random_state=1)
    #model2 = 
    model2 = svm.SVC(kernel='linear', class_weight='balanced', probability=True) # gamma='scale', C = 1
    space = dict()
    space['C'] = np.arange(0.1,2,0.1).tolist()#[1]#[0.1,0.2,0.3,1,2,3,4] #np.arange(0.1,2,0.1).tolist()#[0.1,0.2,0.3,1,2,3,4]#np.arange(2**(-10),0.1,2*(10)).tolist()
    # c_steps = [10e-1, 10e0, 10e1, 10e2]
    
    #space['gamma'] = ['auto','scale']
    #space['max_features'] = [2, 4, 6]
    # define search
    search = GridSearchCV(model2,space, cv=cv_inner, refit=True)
    result = search.fit(x_train_sfs, y_train)
    #print(sfs1.estimator_.coef_)
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_
    # evaluate model on the hold out dataset
    yhat = best_model.predict(x_test_sfs)
    
    model2.fit(x_train_sfs, y_train)
    #print(model2.coef_.shape)
    #feature_ranking[iter_,np.argwhere(sfs1.ranking_==1).ravel()] = np.abs(model2.coef_)
    iter_ = iter_+1

    #prediction = model2.predict_proba(x_test_sfs)
    prediction = best_model.predict_proba(x_test_sfs)
    fpr, tpr, t = roc_curve(y_test.reshape(-1,1), prediction[:, 1])
    interp_tpr = np.interp(mean_fpr, fpr, tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    #plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

    #yhat = model2.predict(x_test_sfs)
    #yhat = best_model.predict(x_test_sfs)
    acc = accuracy_score(y_test, yhat)    
    acc2 = recall_score(y_test, yhat)
    acc3 = precision_score(y_test, yhat)
    tn, fp, fn, tp = confusion_matrix(y_test, yhat).ravel()
    specificity.append(tn/(tn+fp)) 
    tns.append(tn), fps.append(fp), fns.append(fn), tps.append(tp)

    outer_results.append(acc)
    outer_results2.append(acc2)
    outer_results3.append(acc3)
    # report progress     
    # print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
    print('>acc=%.3f acc2=%.3f acc3=%.3f' % (acc,acc2,acc3))
    #print(test_ix)
    #print(test_ix[np.argwhere(y_test!=yhat)])
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
print('Recall: %.3f (%.3f)' % (mean(outer_results2), std(outer_results2)))
print('Precision: %.3f (%.3f)' % (mean(outer_results3), std(outer_results3)))

plt.plot([0,1],[0,1],linestyle = '--',lw = 6,color = 'r',alpha=0.8)
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std = np.std(aucs)

plt.plot(mean_fpr, mean_tpr, color='blue',
        label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std),lw=6, alpha=0.8)

std_tpr = np.std(tprs, axis=0)
#tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
#tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
#plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.2,label=r'$\pm$ 1 std. dev.')
#sns.set_style({'axes.facecolor': '0.9'})
plt.xlabel('False Positive Rate',fontsize = 40)
plt.ylabel('True Positive Rate',fontsize = 40)
plt.title('Cross-Validation ROC of SVM',fontsize = 40)
plt.legend(loc="lower right", prop={'size': 40})
plt.xlim([-0.01, 1.01]) 
plt.ylim([-0.01, 1.01]) 
#plt.xticks(fontsize=16)

plt.yticks(fontsize=40)
plt.xticks(fontsize=40)
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 4
plt.show()
plt.savefig('final.png')

savetxt('mean_tpr_Mdn_knn20Imp.csv', mean_tpr, delimiter=',')
savetxt('mean_fpr_Mdn_knn20Imp.csv', mean_fpr, delimiter=',')
savetxt('mean_auc_Mdn_knn20Imp.csv', mean_auc.reshape(1,-1),delimiter=',')
savetxt('std_Mdn_knn20Imp.csv', std.reshape(-1,1),delimiter=',')

In [23]:
print('Accuracy=',mean(outer_results), 'Sensitivity=',mean(outer_results2)) # mean profile, median interpolation, 1000 repeat, 5 feature, adc, rd, fa
print('AUC=',mean_auc, 'Specificity=',mean(specificity))
print((np.sum(feature_ranking, axis=0)))

Accuracy= 0.7417079710144928 Sensitivity= 0.7352592592592594
AUC= 0.7694493751581968 Specificity= 0.7458342857142857
[  83.   38. 7500. 7455. 5857.   72. 6381.  182. 2113.   89. 7416.  314.]


In [ ]:
mean_tpr_Mdn_knn20Imp = loadtxt('mean_tpr_Mdn_knn20Imp.csv', delimiter=',')
mean_fpr_Mdn_knn20Imp = loadtxt('mean_fpr_Mdn_knn20Imp.csv', delimiter=',')
mean_auc_Mdn_knn20Imp = loadtxt('mean_auc_Mdn_knn20Imp.csv', delimiter=',')
std_Mdn_knn20Imp = loadtxt('std_Mdn_knn20Imp.csv', delimiter=',')


plt.figure(figsize=(20,20))
plt.plot([0,1],[0,1],linestyle = '--',lw = 6,color = 'r',alpha=0.8)
plt.plot(mean_fpr_Mdn_knn20Imp, mean_tpr_Mdn_knn20Imp, color='blue',
        label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc_Mdn_knn20Imp, std_Mdn_knn20Imp),lw=6, alpha=0.8)
#std_tpr = np.std(tprs, axis=0)
plt.xlabel('False Positive Rate',fontsize = 40)
plt.ylabel('True Positive Rate',fontsize = 40)
plt.title('Cross-Validation ROC of SVM',fontsize = 40)
plt.legend(loc="lower right", prop={'size': 40})
plt.xlim([-0.01, 1.01]) 
plt.ylim([-0.01, 1.01]) 

plt.yticks(fontsize=40)
plt.xticks(fontsize=40)
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 4
plt.show()
plt.savefig('final.png')